In [1]:
PART_1_SKIP = True
PART_2_SKIP = True
PART_3_SKIP = True

In [3]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

import skimage
from PIL import Image
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.util import crop, pad
from skimage.morphology import label
from skimage.color import rgb2gray, gray2rgb

import os

import zipfile
z = zipfile.ZipFile('Dancer_Images.zip', 'w')

In [4]:
cap = cv2.VideoCapture('/home/aman/Downloads/dancing_silhouette.mp4')
print(cap.get(cv2.CAP_PROP_FPS))

29.97003017816262


In [10]:
%%time

if PART_1_SKIP == False:
    try:
        if not os.path.xists('data'):
            os.makedirs('data')
        except OSError:
            print('Error: Creating Directory of Data')
            
    currentFrame = 0
    count = 0
    TRAIN_SIZE = 27000
    FRAME_SKIP = 2
    IMG_HEIGHT = 96
    IMG_WIDTH = 64
    IMG_CHANNELS = 1
    X_train = np.zeros((TRAIN_SIZE, IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype='float32')
    video = cv2.VideoWriter('Shadow_Dancer.avi', cv2.VideoWriter_fourcc(*'MJPG'), 30, (IMG_WIDTH, IMG_HEIGHT), False)
    
    while(count < TRAIN_SIZE):
        try:
            ret, frame = cap.read()
            if currentFrame % FRAME_SKIP == 0:
                count += 1
                if count % int(TRAIN_SIZE/10) == 0:
                    print(str(count/TRAIN_SIZE * 100) + "% done")
                ## Preprocessing frame
                img = frame
                img = rgb2gray(img)
                img = resize(img, (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), mode='constant', preserve_range=True)
        
    

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.34 µs
